In [5]:
pip install pyspark


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
from pyspark import SparkConf, SparkContext
import os


from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("streaming").getOrCreate()

#sc.stop()
conf = SparkConf().setMaster("local").setAppName("streaming")
sc = SparkContext.getOrCreate()

In [7]:
static = spark.read.json("data/activity-data/")
dataSchema = static.schema

AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/Users/sandeepkumar/Desktop/data/activity-data.

In [8]:
dataSchema

StructType([StructField('Arrival_Time', LongType(), True), StructField('Creation_Time', LongType(), True), StructField('Device', StringType(), True), StructField('Index', LongType(), True), StructField('Model', StringType(), True), StructField('User', StringType(), True), StructField('gt', StringType(), True), StructField('x', DoubleType(), True), StructField('y', DoubleType(), True), StructField('z', DoubleType(), True)])

In [3]:
static.take(1)

[Row(Arrival_Time=1424686734992, Creation_Time=1424688581040070924, Device='nexus4_2', Index=5, Model='nexus4', User='g', gt='stand', x=-0.0003814697, y=0.025878906, z=0.023727417)]

In [4]:
static.count()

390060

In [5]:
## streaming version
streaming = spark.readStream.schema(dataSchema)\
.option("maxFilesPerTrigger", 1)\
.json("data/activity-data")

In [10]:
activityCounts = streaming.groupBy("gt").count()
activityQuery = activityCounts.writeStream \
.queryName("activity_counts_22")\
.format("memory")\
.outputMode("complete")\
.start()

24/12/23 16:50:07 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/qd/gsytx6pn74s2pb_y9l226t800000gn/T/temporary-d0d04bfb-d0a4-448b-a2b1-cd759b9e014d. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/12/23 16:50:07 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [11]:
from time import sleep
for x in range(7000):
    spark.sql("SELECT * FROM activity_counts_22").show()
    sleep(1)

activityQuery.awaitTermination()



+----------+-----+
|        gt|count|
+----------+-----+
|  stairsup|10457|
|       sit|12311|
|     stand|11386|
|      walk|13254|
|      bike|10795|
|stairsdown| 9363|
|      null|10446|
+----------+-----+



+----------+-----+
|        gt|count|
+----------+-----+
|  stairsup|20915|
|       sit|24622|
|     stand|22771|
|      walk|26508|
|      bike|21590|
|stairsdown|18725|
|      null|20893|
+----------+-----+



+----------+-----+
|        gt|count|
+----------+-----+
|  stairsup|31367|
|       sit|36930|
|     stand|34156|
|      walk|39764|
|      bike|32388|
|stairsdown|28090|
|      null|31341|
+----------+-----+



ERROR:root:KeyboardInterrupt while sending command.                 (0 + 0) / 1]
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/local/Cellar/python@3.10/3.10.9/Frameworks/Python.framework/Versions/3.10/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
static.take(1)

In [ ]:
deviceModelStats = streaming.cube("gt", "model").avg()\
.drop("avg(Arrival_time)")\
.drop("avg(Creation_Time)")\
.drop("avg(Index)")\
.writeStream.queryName("device_counts_11").format("memory")\
.outputMode("complete")\
.start()


from time import sleep
for x in range(5000):
    spark.sql("SELECT * FROM device_counts_11").show()
    sleep(1)

deviceModelStats.awaitTermination()

24/12/23 16:53:02 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/qd/gsytx6pn74s2pb_y9l226t800000gn/T/temporary-15c08551-b38d-418d-8d7f-856578872341. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/12/23 16:53:02 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


+---+-----+------+------+------+
| gt|model|avg(x)|avg(y)|avg(z)|
+---+-----+------+------+------+
+---+-----+------+------+------+

+---+-----+------+------+------+
| gt|model|avg(x)|avg(y)|avg(z)|
+---+-----+------+------+------+
+---+-----+------+------+------+



+---+-----+------+------+------+
| gt|model|avg(x)|avg(y)|avg(z)|
+---+-----+------+------+------+
+---+-----+------+------+------+

+---+-----+------+------+------+
| gt|model|avg(x)|avg(y)|avg(z)|
+---+-----+------+------+------+
+---+-----+------+------+------+



+---+-----+------+------+------+
| gt|model|avg(x)|avg(y)|avg(z)|
+---+-----+------+------+------+
+---+-----+------+------+------+



+---+-----+------+------+------+
| gt|model|avg(x)|avg(y)|avg(z)|
+---+-----+------+------+------+
+---+-----+------+------+------+



...

+---+-----+------+------+------+
| gt|model|avg(x)|avg(y)|avg(z)|
+---+-----+------+------+------+
+---+-----+------+------+------+



+---+-----+------+------+------+
| gt|model|avg(x)|avg(y)|avg(z)|
+---+-----+------+------+------+
+---+-----+------+------+------+



+---+-----+------+------+------+
| gt|model|avg(x)|avg(y)|avg(z)|
+---+-----+------+------+------+
+---+-----+------+------+------+



+----------+------+--------------------+--------------------+--------------------+
|        gt| model|              avg(x)|              avg(y)|              avg(z)|
+----------+------+--------------------+--------------------+--------------------+
|      NULL|nexus4|-3.94516756060561...|-0.00476322271035229|-0.01131527405524...|
|      null|  NULL|-0.01145182049817...|-0.00432467328008807|0.002828257253991...|
|stairsdown|  NULL| 0.02086770352612356|-0.03134959187262...| 0.11766223035735324|
|       sit|  NULL|-5.52516645747706...|2.716525435464201E-4|6.721253376654944E-5|
|stairsdown|nexus4| 0.02086770352612356|-0.03134959187262...| 0.11766223035735324|
|      bike|nexus4|0.017769288779851736|-0.00517382921819...| -0.0841825979213531|
|      null|nexus4|-0.01145182049817...|-0.00432467328008807|0.002828257253991...|
|      walk|nexus4|-0.00285431687661...|0.005146564768930...|-8.78464908201330...|
|      walk|  NULL|-0.00285431687661...|0.005146564768930...|-8.78464908201330...|
|  s

+----------+------+--------------------+--------------------+--------------------+
|        gt| model|              avg(x)|              avg(y)|              avg(z)|
+----------+------+--------------------+--------------------+--------------------+
|      NULL|nexus4|-3.94516756060561...|-0.00476322271035229|-0.01131527405524...|
|      null|  NULL|-0.01145182049817...|-0.00432467328008807|0.002828257253991...|
|stairsdown|  NULL| 0.02086770352612356|-0.03134959187262...| 0.11766223035735324|
|       sit|  NULL|-5.52516645747706...|2.716525435464201E-4|6.721253376654944E-5|
|stairsdown|nexus4| 0.02086770352612356|-0.03134959187262...| 0.11766223035735324|
|      bike|nexus4|0.017769288779851736|-0.00517382921819...| -0.0841825979213531|
|      null|nexus4|-0.01145182049817...|-0.00432467328008807|0.002828257253991...|
|      walk|nexus4|-0.00285431687661...|0.005146564768930...|-8.78464908201330...|
|      walk|  NULL|-0.00285431687661...|0.005146564768930...|-8.78464908201330...|
|  s